In [129]:
import numpy as np

class RNN:
    def __init__(self, input_dim, hidden_dim, output_dim):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        

        # 初始化权重和偏差
        self.W_h = np.random.randn(hidden_dim, hidden_dim) * np.sqrt(2.0 / (hidden_dim + hidden_dim))
        self.W_x = np.random.randn(input_dim, hidden_dim) * np.sqrt(2.0 / (input_dim + hidden_dim))
        self.W_y = np.random.randn(hidden_dim, output_dim) * np.sqrt(2.0 / (hidden_dim + output_dim))
        self.b_h = np.zeros((1, hidden_dim))
        self.b_y = np.zeros((1, output_dim))
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    
    def forward(self, x):
        # 计算隐藏状态
        self.h = np.zeros((1, self.hidden_dim))
        self.hidden_states = []
        for t in range(len(x)):
            self.h = self.sigmoid(np.dot(x[t], self.W_x) + np.dot(self.h, self.W_h) + self.b_h)
            self.hidden_states.append(self.h)
        # 计算输出
        self.y = np.dot(self.h, self.W_y) + self.b_y
        return self.y
    
    def backward(self, x, y_true, learning_rate, clip_norm):
        # 计算输出误差
        d_y = y_true - self.y
    
        # 计算隐藏状态误差
        d_h = np.zeros((1, self.hidden_dim))
        d_W_y = np.zeros_like(self.W_y)
        d_W_h = np.zeros_like(self.W_h)
        d_W_x = np.zeros_like(self.W_x)
        d_b_y = np.zeros_like(self.b_y)
        d_b_h = np.zeros_like(self.b_h)
        for t in reversed(range(len(x))):
            d_W_y += np.outer(self.hidden_states[t], d_y)
            d_b_y += d_y
            d_h = np.dot(d_y, self.W_y.T) + np.dot(d_h, self.W_h.T)
            d_h_raw = (1 - self.hidden_states[t] ** 2) * d_h
            d_W_x += np.outer(x[t], d_h_raw)
            d_W_h += np.outer(self.hidden_states[t-1], d_h_raw)
            d_b_h += d_h_raw
    
        # 对梯度进行裁剪
        grad_norm = np.sqrt(np.sum(d_W_y ** 2) + np.sum(d_W_h ** 2) + np.sum(d_W_x ** 2))
        if grad_norm > clip_norm:
            d_W_y *= clip_norm / grad_norm
            d_W_h *= clip_norm / grad_norm
            d_W_x *= clip_norm / grad_norm
        
        # 更新权重和偏差
        self.W_h -= learning_rate * d_W_h
        self.W_x -= learning_rate * d_W_x
        self.W_y -= learning_rate * d_W_y
        self.b_h -= learning_rate * d_b_h
        self.b_y -= learning_rate * d_b_y


In [139]:
# 准备训练数据
X = np.array([[[0], [1], [0]], [[1], [0], [1]], [[1], [1], [0]], [[0], [0], [1]]])
y_true = np.array([[1], [0], [0], [1]])

# 初始化RNN模型
rnn = RNN(input_dim=1, hidden_dim=10, output_dim=1)

# 训练参数设置
learning_rate = 1e-6
epochs = 1000

# 开始训练
for i in range(epochs):
    loss = 0
    for j in range(len(X)):
        # 前向传播
        y_pred = rnn.forward(X[j])
        loss += np.square(y_true[j] - y_pred)
        # 反向传播
        rnn.backward(X[j], y_true[j], learning_rate,clip_norm=1)
        
    if i % 100== 0:
        print(f"Epoch: {i}, Loss: {loss}")

Epoch: 0, Loss: [[2.56105231]]
Epoch: 100, Loss: [[2.56843361]]
Epoch: 200, Loss: [[2.57585187]]
Epoch: 300, Loss: [[2.58330727]]
Epoch: 400, Loss: [[2.59080001]]
Epoch: 500, Loss: [[2.59833028]]
Epoch: 600, Loss: [[2.60589827]]
Epoch: 700, Loss: [[2.61350417]]
Epoch: 800, Loss: [[2.62114819]]
Epoch: 900, Loss: [[2.62883051]]
